In [1]:
import sys
import os
import numpy as np

path = 'FacialLandmarks/BU4DFE_BND_V1.1/'
X=[]
Y=[]
# 101 faces - sub folders, 6 emotions - Angry, Disgust, Fear, Happy, Sad, Surprise
for face_dir in os.listdir(path)[1:]:
    face_path = path+face_dir+'/'
    for label in os.listdir(face_path):
        emotion_path = face_path+label+'/'
        for file in os.listdir(emotion_path):
            if file.endswith(".bnd") or file.endswith(".landmark"):
                file_path = emotion_path+file
                points = np.loadtxt(emotion_path+file, usecols=(1, 2, 3), encoding='utf-8') #x,y,z
                X.append(points)
                Y.append(label)
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape[0], -1)               
x_length = len(X)
y_length = len(Y)


In [2]:
from math import acos
import math

pi=round(2*acos(0.0), 3)

def translate_data(X):
    X_translated =[]
    for face in X:
        mean = np.mean(face,axis=0)
        print(mean)
        translated_face = face - mean
        X_translated.append(translated_face)
    X_translated = np.array(X_translated)
    return X_translated

def rotate_data(X, axis='x'):
    sin_angle = math.sin(pi) #0
    cos_angle = math.cos(pi) #-1
    rotated_X=[]
    rotated_face = []
    if axis=='x':
        for face in X:
            for point in face:
                rotated_face.append([point[0], cos_angle * point[1] - sin_angle * point[2], sin_angle * point[1] + cos_angle * point[2]])
            rotated_X.append(rotated_face)
    if axis=='y':
        for face in X:
            for point in face:
                rotated_face.append([cos_angle * point[0] + sin_angle * point[2], point[1], -sin_angle * point[0] + cos_angle * point[2]])
            rotated_X.append(rotated_face)
    if axis=='z':
        for face in X:
            for point in face:
                rotated_face.append([cos_angle * point[0] - sin_angle * point[1], sin_angle * point[0] + cos_angle * point[1], point[2]])
            rotated_X.append(rotated_face)
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'")
    rotated_X=np.array(rotated_X)
    return rotated_X


In [3]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_validate

def classification(X,Y, classifier_type='RF'):
    # Select the classifier
    if classifier_type == 'RF':
        print("RF running")
        clf = RandomForestClassifier()
    elif classifier_type == 'SVM':
        clf = SVC()
    elif classifier_type == 'TREE':
        clf = DecisionTreeClassifier()

  # Define the scoring metrics
    scoring = ['accuracy', 'precision_macro', 'recall_macro']
    Y_pred = []
    test_indices = []
    cv = StratifiedKFold(n_splits=10)
    for (train,test) in cv.split(X,Y):
        # print(train,test)
        # print(len(train),len(test))
        # print(X[train],Y[train])
        clf.fit(X[train],Y[train])
        Y_pred.append(clf.predict(X[test]))
        test_indices.append(test)
    
    return Y,Y_pred,test_indices
    # scores = cross_validate(clf, data, labels, cv=cv, scoring=scoring, return_train_score=False)

    # # Since cross_validate returns a dictionary, you can process it however you need.
    # # For example, to print the mean scores:
    # for score_name, score_values in scores.items():
    #     print(f"{score_name}: {np.mean(score_values)}")

    # return scores

In [4]:
y_original,y_prediction,testi = classification(np.array(X),np.array(Y),'TREE')

In [35]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_val_score

def PrintEvalMetrics(pred, indices, y):
    finalPredictions = []
    groundTruth = []
    precisions = []
    recalls = []
    accuracies = []
    for array in indices:
        temp=[]
        for ele in array:
            temp.append(y[ele])
        groundTruth.append(temp)
        #finalPredictions.extend(p)
    for i in range(10):
        print(confusion_matrix(pred[i],groundTruth[i]))
        precisions.append(precision_score(groundTruth[i], pred[i], average='macro'))
        recalls.append(recall_score(groundTruth[i], pred[i], average='macro'))
        accuracies.append(accuracy_score(groundTruth[i], pred[i]))
        # #print(pred[i],y[i])
        # groundTruth.extend(y[i])
    a = 0
    for i in range(len(finalPredictions)):
        if(finalPredictions[i]==groundTruth[i]):
            a+=1
    print(a)
    #print(finalPredictions)
    print(confusion_matrix(finalPredictions, groundTruth))
    print("Precision: ", np.mean(np.array(precisions)))#precision_score(groundTruth, finalPredictions, average='macro'))
    print("Recall: ", np.mean(np.array(recalls)))# recall_score(groundTruth, finalPredictions, average='macro'))
    print("Accuracy: ", np.mean(np.array(accuracies)))# accuracy_score(groundTruth, finalPredictions))

In [36]:
PrintEvalMetrics(y_prediction,testi,y_original)

[[375 117 172 181 207  57]
 [ 81 216 193 220 114 237]
 [141 120 188 106 287 117]
 [ 90 265  59 140  59 137]
 [222  70 298 188 170  90]
 [104 229  94 163 177 368]]
[[318  54 104  74 126  39]
 [ 28 231 110  32 139  80]
 [ 84 147 271 204 143 139]
 [148 149 234 452 189  53]
 [185 114  79  28 257  65]
 [250 322 206 208 160 630]]
[[328  92 153 116  92  88]
 [137 357  96  66  68 161]
 [113 173 185 507 184 247]
 [112 194 204 183 365  92]
 [237  90 181  71 270  94]
 [ 86 111 185  54  36 323]]
[[287 393 167 231 385 189]
 [123 307 435 174 190 288]
 [231  66 128  95 104  96]
 [238 102  88 364 114  96]
 [133  52  41  47  84   4]
 [  1  97 145  86 138 332]]
[[451 203 273  95  87 116]
 [132 217 259  46 178  98]
 [ 82 139 178 219 167  65]
 [ 63 122 113 488 177  42]
 [170 190  89  34 132  55]
 [114 147  92 115 273 630]]
[[357 162 363 342 307 163]
 [166 236 100  82  50 205]
 [107  45 181  97  88  28]
 [133 345 125 231 276  77]
 [160 156  25 152 223  42]
 [ 89  73 211  93  70 491]]
[[298 153 105 170 181 